In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import torch.nn as nn

(train_images, train_labels),(test_images, test_labels)=mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
train_images=train_images.reshape([-1,784]).astype('float32')/255.0
test_images=test_images.reshape([-1,784]).astype('float32')/255.0
train_labels=tf.one_hot(train_labels, depth=10) # size 10짜리 one-hot encoding 만듦
test_labels=tf.one_hot(test_labels, depth=10)
batch_size=1000
train_data=tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data=train_data.repeat().shuffle(5000).batch(batch_size) # 데이터를 shuffle 하여야 고른 분포를 가짐

nH1=256
nH2=256
nH3=256

class class_SoftmaxNN_LC_A(nn.Module): # class 는 항상 pytorch로 만들었기 때문
  def __init__(self):
    self.W_1=tf.Variable(tf.random.normal([784,nH1]))
    self.b_1=tf.Variable(tf.random.normal([nH1]))
    self.W_2=tf.Variable(tf.random.normal([nH1,nH2]))
    self.b_2=tf.Variable(tf.random.normal([nH2]))
    self.W_3=tf.Variable(tf.random.normal([nH2,nH3]))
    self.b_3=tf.Variable(tf.random.normal([nH3]))
    self.W_o=tf.Variable(tf.random.normal([nH3,10]))
    self.b_o=tf.Variable(tf.random.normal([10]))
  def __call__(self,x):
    H_1=tf.nn.relu(tf.matmul(x,self.W_1)+self.b_1)
    H_2=tf.nn.relu(tf.matmul(H_1,self.W_2)+self.b_2)
    H_3=tf.nn.relu(tf.matmul(H_2,self.W_3)+self.b_3)
    Out=tf.matmul(H_3,self.W_o)+self.b_o
    return Out

class class_SoftmaxNN_LC_B(nn.Module): # class 는 항상 pytorch로 만들었기 때문
  def __init__(self):
    self.W_1=tf.Variable(tf.random.normal([784,nH1]))
    self.b_1=tf.Variable(tf.random.normal([nH1]))
    self.W_2=tf.Variable(tf.random.normal([nH1,nH2]))
    self.b_2=tf.Variable(tf.random.normal([nH2]))
    self.W_3=tf.Variable(tf.random.normal([nH2,nH3]))
    self.b_3=tf.Variable(tf.random.normal([nH3]))
    self.W_o=tf.Variable(tf.random.normal([nH3,10]))
    self.b_o=tf.Variable(tf.random.normal([10]))
  def __call__(self,x):
    H_1=tf.nn.relu(tf.matmul(x,self.W_1)+self.b_1)
    H_2=tf.nn.relu(tf.matmul(H_1,self.W_2)+self.b_2)
    H_3=tf.nn.relu(tf.matmul(H_2,self.W_3)+self.b_3)
    Out=tf.matmul(H_3,self.W_o)+self.b_o
    return Out

model_SoftmaxNN_LC_A=class_SoftmaxNN_LC_A()
model_SoftmaxNN_LC_B=class_SoftmaxNN_LC_B()
optimizer=tf.optimizers.Adam(0.01)

def cost_Softmax(x,y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x,labels=y))

def train_optimization_A(MNIST_images,MNIST_labels):
  with tf.GradientTape() as g:
    prediction=model_SoftmaxNN_LC_A(MNIST_images)
    cost=cost_Softmax(prediction,MNIST_labels)
  gradients=g.gradient(cost,vars(model_SoftmaxNN_LC_A).values())
  tf.optimizers.Adam(0.01).apply_gradients(zip(gradients,vars(model_SoftmaxNN_LC_A).values()))
def train_optimization_B(MNIST_images,MNIST_labels):
  with tf.GradientTape() as g:
    prediction=model_SoftmaxNN_LC_B(MNIST_images)
    cost=cost_Softmax(prediction,MNIST_labels)
  gradients=g.gradient(cost,vars(model_SoftmaxNN_LC_B).values())
  tf.optimizers.Adam(0.01).apply_gradients(zip(gradients,vars(model_SoftmaxNN_LC_B).values()))

for epoch in range(1,11):
  for step, (batch_images,batch_labels) in enumerate(train_data.take(60),1):
    train_optimization_A(batch_images,batch_labels)
    if step%10==0:
      pred=tf.nn.softmax(model_SoftmaxNN_LC_A(batch_images))
      accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
      print('Epoch_A: {} \t Step_A: {} \t Accuracy_A: {}'.format(epoch,step,accuracy))
for epoch in range(1,11):
  for step, (batch_images,batch_labels) in enumerate(train_data.take(60),1):
    train_optimization_B(batch_images,batch_labels)
    if step%10==0:
      pred=tf.nn.softmax(model_SoftmaxNN_LC_B(batch_images))
      accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
      print('Epoch_B: {} \t Step_B: {} \t Accuracy_B: {}'.format(epoch,step,accuracy))

print('Testing_A')
test_model=tf.nn.softmax(model_SoftmaxNN_LC_A(test_images))
test_accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
print('Test Accuracy_A:', test_accuracy.numpy(), '\n')

print('Testing_B')
test_model=tf.nn.softmax(model_SoftmaxNN_LC_A(test_images))
test_accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
print('Test Accuracy_B:', test_accuracy.numpy())

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
Epoch_A: 1 	 Step_A: 10 	 Accuracy_A: 0.9193999767303467
Epoch_A: 1 	 Step_A: 20 	 Accuracy_A: 0.9508000016212463
Epoch_A: 1 	 Step_A: 30 	 Accuracy_A: 0.9539999961853027
Epoch_A: 1 	 Step_A: 40 	 Accuracy_A: 0.9628999829292297
Epoch_A: 1 	 Step_A: 50 	 Accuracy_A: 0.9661999940872192
Epoch_A: 1 	 Step_A: 60 	 Accuracy_A: 0.9736999869346619
Epoch_A: 2 	 Step_A: 10 	 Accuracy_A: 0.9793000221252441
Epoch_A: 2 	 Step_A: 20 	 Accuracy_A: 0.9785000085830688
Epoch_A: 2 	 Step_A: 30 	 Accuracy_A: 0.9818000197410583
Epoch_A: 2 	 Step_A: 40 	 Accuracy_A: 0.9818999767303467
Epoch_A: 2 	 Step_A: 50 	 Accuracy_A: 0.9847000241279602
Epoch_A: 2 	 Step_A: 60 	 Accuracy_A: 0.986299991607666
Epoch_A: 3 	 Step_A: 10 	 Accuracy_A: 0.9861000180244446
Epoch_A: 3 	 Step_A: 20 	 Accuracy_A: 0.986299991607666
Epoch_A: 3 	 Step_A: 30 	 Accuracy_A: 0.9907000064849854
Epoch_A: 3 	 Step_A: 40 	 Accura